In [2]:
import torch
from model import GPTConfig, GPT, QLinearPerChannel, QLinearTile2D

In [3]:
ckpt_path = 'out-shakespeare-char/ckpt.pt'
checkpoint = torch.load(ckpt_path, map_location='cpu')

model_args = checkpoint['model_args']

config = GPTConfig(**model_args)
model = GPT(config)

number of parameters: 10.67M


In [4]:
state_dict = checkpoint['model']
unwanted_prefix = '_orig_mod.'
for k in list(state_dict.keys()):
    if k.startswith(unwanted_prefix):
        state_dict[k[len(unwanted_prefix):]] = state_dict.pop(k)
model.load_state_dict(state_dict)

<All keys matched successfully>

In [ ]:
total_zeros_weight = 0
total_params_weight = 0

total_below_eps = 0
total_b_entries = 0
eps = 1e-6

for module in model.modules():
    if isinstance(module, (QLinearPerChannel, QLinearTile2D)):
        weight = module.weight.detach()
        total_zeros_weight += (weight.abs() < eps).sum().item()
        total_params_weight += weight.numel()

        b = module.b.detach()
        total_below_eps += (b < eps).sum().item()
        total_b_entries += b.numel()

sparsity_weights = total_zeros_weight / total_params_weight
sparsity_b = total_below_eps / total_b_entries


print(f"Total zero weights: {total_zeros_weight} / {total_params_weight} "
      f"({sparsity_weights:.4f} sparsity)")

print(f"Total b < {eps}: {total_below_eps} / {total_b_entries} "
      f"({sparsity_b:.4f} b sparsity)")

ZeroDivisionError: division by zero